test cells continuously?


In [ ]:
var importer = require('../Core');
var path = require('path');
var Module = require('module').Module;
var Mocha = require('mocha');

// call test runner
var mocha;
if(typeof mocha === 'undefined') {
    mocha = new Mocha({
        ui: 'bdd',
        reporter: 'list'
    });
}

function resetTests(suite) {
    suite.tests.forEach(function(t) {
        delete t.state;
        t.timedOut = false;
    });
    suite.suites.forEach(resetTests);
    suite.tests = [];
}

var testCells = (cells, ctx = {}) => {
    cells.forEach(r => {
        mocha.suite.emit('pre-require', ctx, r.id, mocha);
        Object.assign(ctx, Mocha);
        var required = r.runInNewContext(ctx);
        Object.assign(ctx, required);
        mocha.suite.emit('require', required, r.id, mocha);
        mocha.suite.emit('post-require', ctx, r.id, mocha);    
    });
    return new Promise(resolve => mocha.run(function (failures) {
        resetTests(mocha.suite);
        resolve(failures);
    }));
}
module.exports = testCells;


watch files run tests?


In [ ]:
var importer = require('../Core');
var testCells = importer.import('test cells continuously');
var watch = require('glob-watcher');

var callTests = (tests) => {
    try {
        var cells = importer.interpret(typeof tests === 'string' ? [tests] : tests);
        return testCells(cells);
    } catch (e) {
        console.log(e);
    }
}

// TODO: code analysis to combine blocks into modules?

var rateLimiter;
var testWatcher = (files, tests) => {
    files = typeof files === 'string' ? [files] : files;
    watch(['../**/*.ipynb'], () => {
//    watch(['../!(\.*|node_modules)**/*'], () => {
        console.log('running all tests');
        return callTests(tests)
    });
    callTests(tests);
    var stdin = process.openStdin();
    stdin.addListener("data", function(d) {
        stdin.close();
    });
    // TODO: restart this test script every loop? fork and abandon current thread?
    // TODO: update cell cache in intrepret notebooks.ipynb
    // TODO: git apply without whitespace, reset the rest?  separate index?  how do make git-scenario app?
    // TODO: re-import cells on Utilities/.modules folder change. run all "watcher" commands from a test below?  mocha.grep?
}
module.exports = testWatcher;


test test runner?



In [ ]:
var importer = require('../Core');
var path = require('path');

try {
    var r = importer.import([
        'test cells continuously',
        'list project files'
    ]);
    var {testCells, listInProject} = r;

    var files = listInProject(path.dirname(__dirname), '**/*.ipynb');
    var cells = files.reduce((cells, f) => {
        try {
            const notebookCells = importer.interpret(path.basename(f));
            const testable = notebookCells
                .filter(c => c.markdown.join('').match(/test/ig)
                        && c.fresher.indexOf('describe(') > -1
                       && c.filename.indexOf('test runner.ipynb') === -1);
            
            const allCellsForNotebook = [];
            
            // include cells leading up to matching cell, concat ... indexOf ...
            testable.forEach(c => {
                const leadingCells = notebookCells.slice(0, notebookCells.indexOf(c) + 1)
                // TODO: not necessary, no questions means it's unaddressable by matching
                    .filter(c => c.questions.length > 0)
                leadingCells.forEach(c => allCellsForNotebook.push(c));
            });
            
            cells = cells.concat(allCellsForNotebook.filter((c, i, arr) => arr.indexOf(c) === i));
        } catch (e) {
            if((e + '').indexOf('Nothing found') > -1) {
                return cells; // ignore it
            }
            throw e;
        }
        return cells;
    }, [])
    cells.forEach(r => {
        try {
            var required = r.runInNewContext(global);
            Object.assign(global, required);
        } catch (e) {
            console.log(e);
        }

    });
} catch (e) {
    console.log(e);
}

describe('test runner service', () => {
    it('should find all notebook tests', () => {
        // TODO: automatically turn console logs on and off for tests that fail or have changed
//        console.log(cells.map(c => c.id));
        //testCells(cells).then(r => {
        //    done();
        //})

        // TODO: put them in the order to include them in the context
        
    })
    
    it('should run notebook tests', () => {
        
    })
    
    it('should run all tests when a dependency changes', () => {
        
    })
})

In [ ]:
var importer = require('../Core');
var framework = require('wdio-mocha-framework');
var wdio = require('wdio-sync');
var Module = require('module').Module;
var Mocha = require('mocha');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var INTERFACES = {
    bdd: ['before', 'beforeEach', 'it', 'after', 'afterEach'],
    tdd: ['suiteSetup', 'setup', 'test', 'suiteTeardown', 'teardown'],
    qunit: ['before', 'beforeEach', 'test', 'after', 'afterEach']
}

var mocha;
if(typeof mocha === 'undefined') {
    mocha = new Mocha({
        ui: 'bdd',
        compilers: ['js:babel-register'],
        timeout: 100000,
        reporter: 'list'
    });
}

// put them in the order to include them in the context
var f = PROFILE_PATH + '/Documents/Selenium/ecommerce/test/specs/trial.js';
var ctx = {};
var {testCells, createWebdriverClient} = importer.import([
    'test cells continuously',
    'webdriver client'
], {mocha});

//    const adapter = new framework.MochaAdapter(cid, config, specs, capabilities)
//    return adapter.run()

ctx.browser = ctx.client = createWebdriverClient();
Object.assign(ctx.client.options, {mochaOpts: mocha.options})

var r = {
    id: f,
    runInNewContext: (ctx) => {
        return importer.runInNewContext(fs.readFileSync(f).toString(), Object.assign(ctx, {
            __filename: f
        }))
    }
}

mocha.suite.emit('pre-require', ctx, r.id, mocha);
Object.assign(ctx, Mocha);
mocha.suite.emit('require',
                 Object.assign(ctx, r.runInNewContext(ctx)),
                 r.id, mocha);
mocha.suite.emit('post-require', ctx, r.id, mocha);  

wdio.wrapCommands(
    ctx.client,
    ctx.client.options.beforeCommand,
    ctx.client.options.afterCommand)

mocha.suite.on('pre-require', (context, file, m) => {
    Object.assign(mocha.options, {
        context, file, mocha: m, options: mocha.options
    })

    INTERFACES[mocha.options.ui].forEach((fnName) => {
        let testCommand = INTERFACES[mocha.options.ui][2]

        wdio.runInFiberContext(
            [testCommand, testCommand + '.only'],
            ctx.client.options.beforeHook,
            ctx.client.options.afterHook,
            fnName,
            ctx
        )
    })
})

$$.async();
new Promise(resolve => mocha.run(function (failures) {
    resetTests(mocha.suite);
    resolve(failures);
}))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))
